Teniendo en cuenta los resultados y las conclusiones de los anteriores notebooks, normalizaremos las variables Tempo, Loudness y Duration_ms y transformaremos a logaritmo el target Streams


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

train_df = pd.read_csv('spotify_limpio_train.csv')

train_df.head()

,Artist,Track,Album,Album_type,Stream,Danceability,Energy,Key,Key_name,Loudness,Speechiness,Acousticness,Liveness,Valence,Tempo,Tempo_category,Duration_ms,Licensed,official_video
0,MGMT,Me and Michael,Little Dark Age,album,51859353.0,0.628,0.841,9,A,-4.583,0.0268,0.0285,0.2540,0.706,111.985,Moderato / Allegretto,289853.0,True,True
1,Mc Lipi,Quebradas,Quebradas,single,31545318.0,0.762,0.714,9,A,-2.436,0.2190,0.1460,0.0415,0.558,112.985,Moderato / Allegretto,250588.0,True,True
2,Justin Quiles,La Esquina del Mall,La Esquina del Mall,single,17950239.0,0.701,0.628,6,F# / Gb,-5.026,0.2930,0.2430,0.3330,0.777,189.893,Presto,207388.0,True,True
3,Arctic Monkeys,Snap Out Of It,AM,album,376045969.0,0.728,0.638,5,F,-6.455,0.0336,0.2490,0.1160,0.872,130.014,Allegro,193030.0,True,True
4,MEDUZA,Tell Me Why - MEDUZA Remix,Tell Me Why (MEDUZA Remix),single,22769089.0,0.561,0.928,6,F# / Gb,-5.812,0.0371,0.0870,0.3320,0.182,125.988,Allegro,171429.0,True,True


In [ ]:
train_df = train_df.drop(columns=[ 'Artist', 'Track', 'Album', 'Album_type', 'Key_name', 'Tempo_category'])

In [ ]:
train_df["log_stream"] = np.log1p(train_df["Stream"])

In [ ]:
train_df.head()

,Stream,Danceability,Energy,Key,Loudness,Speechiness,Acousticness,Liveness,Valence,Tempo,Duration_ms,Licensed,official_video,log_stream
0,51859353.0,0.628,0.841,9,-4.583,0.0268,0.0285,0.2540,0.706,111.985,289853.0,True,True,17.764046
1,31545318.0,0.762,0.714,9,-2.436,0.2190,0.1460,0.0415,0.558,112.985,250588.0,True,True,17.266936
2,17950239.0,0.701,0.628,6,-5.026,0.2930,0.2430,0.3330,0.777,189.893,207388.0,True,True,16.703114
3,376045969.0,0.728,0.638,5,-6.455,0.0336,0.2490,0.1160,0.872,130.014,193030.0,True,True,19.745222
4,22769089.0,0.561,0.928,6,-5.812,0.0371,0.0870,0.3320,0.182,125.988,171429.0,True,True,16.940914


In [8]:
test_df = pd.read_csv('spotify_test.csv')
test_df["log_stream"] = np.log1p(test_df["Stream"])

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

cols_to_scale = ["Loudness", "Tempo", "Duration_ms"]

train_df[cols_to_scale] = scaler.fit_transform(
    train_df[cols_to_scale]
)

Random forest ha sido el algoritmo seleccionado, porque nos daba las mejores métricas

In [ ]:
import pandas as pd

X_train = train_df.drop(columns=['Stream', 'log_stream'], axis=1)
y_train = train_df[ 'log_stream']

X_test = test_df.drop(columns=['Stream', 'log_stream'], axis=1)
y_test = test_df['log_stream']

Aqui estamos probando el algoritmo GridSearchCV para encontrar los mejores hiperparámetros

In [19]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV

# Modelo base
rf = RandomForestRegressor(
    random_state=42,
    n_jobs=-1
)

# Espacio de hiperparámetros
param_grid = {
    "n_estimators": np.arange(100, 600, 100),
    "max_depth": [None, 10, 20],
    "min_samples_split": [2, 5, 10]
}

# Grid Search + validación cruzada
grid_search = GridSearchCV(
    estimator=rf,
    param_grid=param_grid,
    cv=5,
    scoring="neg_root_mean_squared_error",
    n_jobs=-1
)

# Entrenamiento
grid_search.fit(X_train, y_train)

# Mejor modelo
best_rf = grid_search.best_estimator_

print("Mejores hiperparámetros:")
print(grid_search.best_params_)

print(f"Mejor RMSE CV: {-grid_search.best_score_:.4f}")

KeyboardInterrupt: 

A continuación vamos a usar el código RandomizedSearchCV para ver los mejores hiperparámetros, para tener otra alternativa y compararlos

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
import numpy as np

rf = RandomForestRegressor(
    random_state=42,
    n_jobs=-1
)

param_dist = {
    "n_estimators": np.arange(100, 600, 100),
    "max_depth": [None, 10, 20, 30],
    "min_samples_split": [2, 5, 10]
}

random_search = RandomizedSearchCV(
    estimator=rf,
    param_distributions=param_dist,
    n_iter=30,
    cv=5,
    scoring="neg_root_mean_squared_error",
    random_state=42,
    n_jobs=-1
)

random_search.fit(X_train, y_train)

best_rf = random_search.best_estimator_

print("Mejores hiperparámetros:")
print(random_search.best_params_)

print(f"Mejor RMSE CV: {-random_search.best_score_:.4f}")

/opt/anaconda3/envs/aprendizajeautomaticocd/lib/python3.13/site-packages/joblib/externals/loky/process_executor.py:782: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


Mejores hiperparámetros:
{'n_estimators': np.int64(500), 'min_samples_split': 2, 'max_depth': None}
Mejor RMSE CV: 1.5060


Ya tenemos los mejores hiperparámetros, a continuación aplicamos el algoritmo Random forest, a todo train

In [10]:
from sklearn.model_selection import cross_val_score
import numpy as np


def val_cruz(model, X, y, n_splits=5, random_state=42):
    

    #cv = KFold(n_splits=n_splits, shuffle=True, random_state=random_state)


    scores_rmse = cross_val_score(
        model,
        X,
        y,
        cv=n_splits,
        scoring="neg_root_mean_squared_error"
    )


    scores_mape = cross_val_score(
            model,
            X,
            y,
            cv=n_splits,
            scoring="neg_mean_absolute_percentage_error"
        )
        
    scores_mae = cross_val_score(
        model,
        X,
        y,
        cv=n_splits,
        scoring="neg_mean_absolute_error"
    )

    scores_r2 = cross_val_score(
        model,
        X,
        y,
        cv=n_splits,
        scoring="r2"
    )

    # Pasar a positivo
    scores_rmse = -scores_rmse
    scores_mae = -scores_mae
    scores_mape = -scores_mape 

    print(" Resultados validación cruzada")
    print("-" * 40)
    print(f"RMSE: {scores_rmse}")
    print(f"MAPE: {scores_mape}")
    print(f"MAE:  {scores_mae}")
    print(f"R²:   {scores_r2}")
    print("-" * 40)
    print(f"RMSE medio: {scores_rmse.mean():.4f} ± {scores_rmse.std():.4f}")
    print(f"MAPE medio:  {scores_mape.mean():.4f} ± {scores_mape.std():.4f}")
    print(f"MAE medio:  {scores_mae.mean():.4f} ± {scores_mae.std():.4f}")
    print(f"R² medio:   {scores_r2.mean():.4f} ± {scores_r2.std():.4f}")

    return scores_rmse.mean(), scores_rmse.std(), scores_mape.mean(), scores_mape.std(), scores_mae.mean(),scores_mae.std(),scores_r2.mean(),scores_r2.std()



Random forest

In [11]:
from sklearn.ensemble import RandomForestRegressor


model = RandomForestRegressor(n_estimators = 500, max_depth = None, min_samples_split = 2, random_state=42)

model.fit(X_train, y_train)

y_pred = model.predict(X_test)


ValueError: The feature names should match those that were passed during fit.
Feature names unseen at fit time:
- Album
- Album_type
- Artist
- Key_name
- Tempo_category
- ...
